In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, 
                             classification_report, roc_curve)

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def catboost_metric(model_cb, X_test, y_test ):
    y_pred_cb = model_cb.predict(X_test)
    y_pred_cb_proba = model_cb.predict_proba(X_test)[:, 1]

    roc_auc = roc_auc_score(y_test, y_pred_cb_proba)
    accuracy = accuracy_score(y_test, y_pred_cb)
    precision = precision_score(y_test, y_pred_cb)
    recall = recall_score(y_test, y_pred_cb)
    f1 = f1_score(y_test, y_pred_cb)

    metrics_cb = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'ROC-AUC': roc_auc
        }
    return metrics_cb, y_pred_cb, y_pred_cb_proba

def lightgbm_metric( model, X_test, y_test):

    y_pred_lgb = model.predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_lgb)   
    best_threshold = thresholds[(tpr - fpr).argmax()]
    print(best_threshold)

    roc_auc = roc_auc_score(y_test, y_pred_lgb)
    print(f'ROC AUC: {roc_auc:.4f}')
    y_pred = (y_pred_lgb >= best_threshold).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    metrics = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'ROC-AUC': roc_auc
        }
    return metrics, y_pred, y_pred_lgb

def plot_confusion_matrix(y_test, y_pred, title='Model'):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                cbar=False)
    plt.title(f'Confusion Matrix - {title}', fontsize=16, fontweight='bold')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()
    return plt


# Загрузка данных

In [ ]:
train = pd.read_csv('train.csv')
train.create_time = pd.to_datetime(train.create_time)
train.model_create_time = pd.to_datetime(train.model_create_time)

test = pd.read_csv('test.csv')
test.create_time = pd.to_datetime(test.create_time)
test.model_create_time = pd.to_datetime(test.model_create_time)

web = pd.read_csv('t1_webstat.csv')
web.sort_values('date_time', inplace=True)
web.date_time = pd.to_datetime(web.date_time)

In [ ]:
display(web, train)

In [ ]:
session_agg = web.groupby('sessionkey_id', sort=False).agg(
    product_sale_count=('product_in_sale', 'sum'),
    total_sec_dt=('date_time', lambda x: (x.iloc[-1] - x.iloc[0]).total_seconds()),
    total_sec_dur=('pageview_duration_sec', 'sum'),
    unique_pages_count=('pageview_number', 'nunique'),
    mean_sec_dur=('pageview_duration_sec', 'mean'),
    max_sec_dur=('pageview_duration_sec', 'max'),
    total_receipt=('price', 'sum') 
)

session_agg['sec_diff'] = session_agg['total_sec_dt'] - session_agg['total_sec_dur']

session_agg